In [ ]:
import dask
import matplotlib.pyplot as plt
import xarray as xr

from dask.distributed import Client
from pathlib import PurePath, Path

CHPC = 'chpc' in str(Path.home())

In [ ]:
home = PurePath(f'{Path.home()}/scratch/iSnobal/runs/run201[7,8]*')
snow_nc = 'snow.nc'

#### CHPC

In [ ]:
if CHPC:
    from dask_jobqueue import SLURMCluster
    
    cluster = SLURMCluster(
        cores=12,
        processes=12,
        project="notchpeak-shared-short",
        queue="notchpeak-shared-short",
        memory='12g'
    )

    from distributed import Client
    from dask import delayed

    cluster.scale(1)
    client = Client(cluster)
else:
    from dask.distributed import LocalCluster
    
    # Assume local
    cluster = LocalCluster(memory_limit='1.5GB')
    client = Client(cluster)


# print(cluster.job_script())
client

In [ ]:
snow = xr.open_mfdataset(
    str(home.joinpath(snow_nc)),
    data_vars=['thickness'],
    parallel=True,
)

In [ ]:
# max = snow.thickness.max().compute()

In [ ]:
def plot_day(snow_day):
    if snow_day.time.dt.hour != 23:
        return
    
    plt.figure(figsize=(10, 10), dpi=150)
    snow_day.plot(cmap=plt.cm.Blues, vmax=3, cbar_kwargs={'label': 'Snow Height'})
    plt.xlabel('')
    plt.ylabel('')
    plt.gca().ticklabel_format(style='plain')
    plt.savefig(f'{Path.home()}/scratch/iSnobal/plots/{str(snow_day.time.dt.strftime("%y%m%d").values)}.jpg')
    plt.close()

In [ ]:
dask_queue = []

for snow_day in snow.thickness:
    dask_queue.append(
        dask.delayed(plot_day)(snow_day)
    )

In [ ]:
dask.compute(dask_queue);

In [ ]:
snow.thickness[-1:].plot(figsize=(10, 10), cmap=plt.cm.Blues, cbar_kwargs={'label': 'Snow Height'})
plt.xlabel('')
plt.ylabel('')
plt.gca().ticklabel_format(style='plain')

### Convert to video using ffmpeg

```shell
ffmpeg -pattern_type glob -i '*.jpg' -crf 18 -filter:v "setpts=2.5*PTS" snow_depth.mp4
```